## Introduction to Machine Learning

   ![](images/ml_map.svg)

## Unsupervised Learning Overview
*Find patterns in data without explicit labels*

* Dimensionality Reduction
* Clustering
* Anomaly Detection


##  Scikit-learn Unsupervised API

`X` : Data matrix (n_samples, n_features)

::: {.incremental}
* `fit(X)`: Learn from the data
* `transform(X)`: Apply the transformation
* `fit_transform(X)`: Learn and apply in one step
* `inverse_transform(X)`: Reverse the transformation
* `predict(X)`: Predict labels or values
* `fit_predict(X)`: Learn and predict in one step
:::

## Introduction to Dimensionality Reduction

*Reducing the number of features (columns) while keeping most of the information*

. . .

`X` (n_samples, n_features) $\rightarrow$  `X'`  (n_samples, n_components)

## Principal Component Analysis (PCA)
* A linear transformation
* Finds the directions of maximum variance in the data
* Very fast and efficient

## PCA Example with scikit-learn


In [ ]:
#| echo: false
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(0)
X = np.random.multivariate_normal(mean=[0, 0], cov=[[1, 0.9], [0.9, 1]], size=100)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [ ]:
#| echo: false

fig, ax = plt.subplots(ncols=2, figsize=(10, 4))
ax[0].scatter(X[:, 0], X[:, 1], label='Original Data')
ax[0].scatter(X_pca[:, 0], X_pca[:, 1], label='Transformed Data')
ax[0].set_xlabel('Feature 1')
ax[0].set_ylabel('Feature 2')
ax[0].legend()

ovar = np.var(X, axis=0)
pvar = np.var(X_pca, axis=0)

# make a bar chart with the variance of each feature in the original and transformed data
ax[1].bar(['Feature 1', 'Feature 2'], ovar, label='Original Data')
ax[1].bar(['PC1', 'PC2'], pvar, label='Transformed Data')
ax[1].set_ylabel('Variance');

## PCA Wine data {.smaller}

In [ ]:
from sklearn.datasets import load_wine
X= load_wine(as_frame=True)['data']
X.head()

---

In [ ]:
#| echo: false

import seaborn as sns

sns.heatmap(X.corr(), cmap='coolwarm', vmin=-1, vmax=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
Xsc = StandardScaler().fit_transform(X)
pca = PCA()
pca.fit(Xsc);

plt.bar(range(1, pca.n_components_+1), pca.explained_variance_)
plt.ylabel('Explained variance')
plt.xlabel('Principal component');

---

In [ ]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

In [ ]:
#| echo: false
fig,ax = plt.subplots()

ax.bar(range(1, 14), loadings[:,0])
ax.set_ylim(-1,1)
ax.set_xticks(range(1,14))
ax.set_xticklabels(X.columns,rotation=90)
ax.set_ylabel('Loadings for PC1');

## Pipelines {.smaller}

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

pipe = make_pipeline(StandardScaler(),
                     PCA(n_components=2),
                     KMeans(n_clusters=3))

pipe

::: {.notes}
* ML workflows consist of multiple steps
* Scikit-learn pipelines simplify the process
* Ensures consistent preprocessing and modeling in training and testing
:::

## PCA with MIKE data

In [ ]:
import mikeio
da = mikeio.read("data/ns_hs.dfsu")[0]
da.isel(time=-1).plot();

In [ ]:
from sklearn.preprocessing import StandardScaler

da.values[np.isnan(da.values)] = 0

pipe = make_pipeline(
                     StandardScaler(),
                     PCA(n_components=50))

Xtr = pipe.fit_transform(da.to_numpy())

In [ ]:
#| echo: false
n_components = pipe.named_steps['pca'].n_components_
plt.bar(range(1,n_components+1), pipe.named_steps['pca'].explained_variance_)
plt.yscale('log')
plt.ylabel('Explained variance')
plt.xlabel('Principal component');

---


In [ ]:
Xrec = pipe.inverse_transform(Xtr)
darec = mikeio.DataArray(Xrec,
                         time=da.time, 
                         item=da.item,
                         geometry=da.geometry)

In [ ]:
# | echo: false
fig, ax = plt.subplots(ncols=2)

da.isel(time=-1).plot(ax=ax[0], title="Original", add_colorbar=False, vmin=0,vmax=0.9);
# remove axis labels and ticks to make a clean plot
ax[0].set_xlabel('')
ax[0].set_ylabel('')
ax[0].set_xticks([])
ax[0].set_yticks([])
darec.isel(time=-1).plot(ax=ax[1], title="Reconstructed", add_colorbar=False, vmin=0,vmax=0.9);
ax[1].set_xlabel('')
ax[1].set_ylabel('')
ax[1].set_xticks([])
ax[1].set_yticks([])

In [ ]:
rel_err = (da - darec)*100 / da
rel_err 

In [ ]:
# | echo: false
timestep = -1
ax = (rel_err.isel(time=timestep)
                             .plot(title="Relative reconstruction error (%)",
                                  cmap='coolwarm', vmin=-2, vmax=2,
                                  label="", levels=9));
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('')
ax.set_ylabel('');

## PCA with MIKE data - Loadings

In [ ]:
# | echo: false

loadings = pipe.named_steps['pca'].components_.T * np.sqrt(pipe.named_steps['pca'].explained_variance_)

n_components = 6

fig, ax = plt.subplots(ncols=6, figsize=(15,4))

for i in range(n_components):
    daloading = mikeio.DataArray(loadings[:,i], geometry=da.geometry)
    daloading.plot(ax=ax[i], title=f"PC{i+1}", add_colorbar=False, vmin=-1, vmax=1,cmap='coolwarm')
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_xlabel('')
    ax[i].set_ylabel('')

## T-Distributed Stochastic Neighbor Embedding (t-SNE)

::: {.incremental}
* Non-linear dimensionality reduction
* Preserves local structure in the data
* Useful for visualization
* Does **not** support inverse transformation
:::

## Digits data

In [ ]:
from sklearn.datasets import load_digits

X, y= load_digits(return_X_y=True)
X.shape

In [ ]:
#| echo: false

# visualize the first 10 digits
fig, axes = plt.subplots(1, 10, figsize=(10, 1))
for i in range(10):
    axes[i].imshow(X[i].reshape(8, 8), cmap='gray')
    axes[i].axis('off')

---

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, init='pca')
X_ts = tsne.fit_transform(X)

In [ ]:
#| echo: false
plt.scatter(X_ts[:, 0], X_ts[:, 1], c=y, cmap='viridis')

## Introduction to Clustering

*Grouping similar data points together*

In [ ]:
#| echo: false
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

X, y = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)
kmeans = KMeans(n_clusters=4)
pred = kmeans.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], s=50, c=pred, cmap='viridis')

## K-Means Clustering

*Identify **K** clusters in the data*
. . .

1. Initialize cluster centroids randomly
2. Assign each data point to the nearest centroid
3. Update the centroids based on the mean of the assigned data points
4. Repeat steps 2 and 3 until convergence

. . .

*Remember to scale the data before applying K-Means*

## K-Means example wine data {.smaller}

In [ ]:
#| echo: false
from sklearn.datasets import load_wine

data_dict = load_wine(as_frame=True)
X = data_dict['data']
y = data_dict['target']
X.head()

In [ ]:
import plotly.express as px

px.scatter_3d(X, x='flavanoids', y='color_intensity', z='alcohol')

---

In [ ]:
pipe = make_pipeline(
    StandardScaler(),
    TSNE(n_components=2, init='pca')
    )

Xtr = pipe.fit_transform(X)

plt.scatter(Xtr[:, 0], Xtr[:, 1])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3)
labels = kmeans.fit_predict(X_scaled)

In [ ]:
#| echo: false
plt.scatter(Xtr[:, 0], Xtr[:, 1], c=labels, cmap='viridis')
plt.show()

---

In [ ]:
pd.crosstab(y, labels, rownames=['True'], colnames=['Cluster'])

## Introduction to Anomaly Detection

*Identify outliers in the data*

In [ ]:
#| echo: false

from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=2, cluster_std=0.20, random_state=0)
X[0] = [1.5, 2.0]
X[1] = [2.5, 3.2]
X[2] = [0.5, 1.0]

from sklearn.ensemble import IsolationForest
model = IsolationForest(contamination=0.01)
y_pred = model.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], s=50, c=y_pred, cmap='viridis')

## Star Wars characters data

In [ ]:
df = pd.read_csv("data/starwars.csv").dropna(subset=['mass', 'height'])
df.head()

In [ ]:
px.scatter(df, x='height', y='mass', hover_name='name')

## Isolation Forest

* Based on decision trees
* Randomly select features, splits data
* Anomalies are isolated in few splits
* Combines results from multiple trees (forest)

## Isolation Forest example

In [ ]:
from sklearn.ensemble import IsolationForest

X = df[['height', 'mass']]

clf = IsolationForest(contamination=0.01)

y_pred = clf.fit_predict(X)
y_pred

In [ ]:
#| echo: false
df['anomaly'] = y_pred
px.scatter(df, x='height', y='mass', color='anomaly', hover_name='name')

. . .

In [ ]:
#| echo: false
df[df['anomaly']==-1][['name', 'height', 'mass', 'anomaly']]

## Summary


::: {.incremental}
* `scikit-learn` provides a wide range of unsupervised learning algorithms
* **Dimensionality reduction** : Extract important features from data, efficiency, compression and visualization
* **Clustering** : Group similar data points together, semi-supervised learning
* **Anomaly detection** : Identify outliers in data
:::